In [ ]:
# IE360 HW2 – Q1

import cvxpy as cp
import numpy as np

# Operation totals
op_demand = np.array([7000, 5500, 4500, 3000], dtype=float)

# Product totals
prod_demand = np.array([2200, 1600, 1400, 1800, 1200], dtype=float)

# costs
machine_cost = np.array([320000, 240000, 280000, 300000], dtype=float)  # M
carrier_cost = np.array([95000,  85000,  75000], dtype=float)           # MH
budget = 10000000

# c_ij: cost to run operation i on machine type j
C_op = np.array([
    [ 8, 11,  9, 14],
    [10,  9,  7, 12],
    [12,  6,  5,  8],
    [15, 13, 11,  6],
], dtype=float)

# h_mn: handling cost for product m on handling system n
C_hand = np.array([
    [ 8, 10, 11],   # basic
    [16,  4, 13],   # high-tech
    [11,  9,  9],   # sports
    [10,  8,  6],   # luxury
    [14, 12,  8],   # premium
], dtype=float)

t_op   = 8.0 / C_op        # time to do 1 unit of op i on machine j
t_hand = 1.0 / C_hand      # time to move 1 unit of product m on handler n
T_avail = 750.0            # unit time

# Decision variables
# v_j: # of machines of type j; w_n: number of handling carriers of type n
v = cp.Variable(4, integer=True, nonneg=True, name="machines")  # M
w = cp.Variable(3, integer=True, nonneg=True, name="carriers")  # MH

# x_ij: amount of operation i processed on machine type j
x = cp.Variable((4, 4), nonneg=True, name="op_assignment")

# y_mn: amount of product m handled by handling system n
y = cp.Variable((5, 3), nonneg=True, name="hand_assignment")

# Constraints
cons = []

# Do all required operations
cons += [cp.sum(x, axis=1) == op_demand]  # for each operation i

# Machine time capacity: sum_i x[i,j] * t_op[i,j] <= 750 * v[j]
for j in range(4):
    cons += [cp.sum(cp.multiply(x[:, j], t_op[:, j])) <= T_avail * v[j]]

# Move all products
cons += [cp.sum(y, axis=1) == prod_demand]  # for each product m

# Handling time capacity: sum_m y[m,n] * t_hand[m,n] <= 750 * w[n]
for n in range(3):
    cons += [cp.sum(cp.multiply(y[:, n], t_hand[:, n])) <= T_avail * w[n]]

# Capital budget
cons += [machine_cost @ v + carrier_cost @ w <= budget]

# Minimize total cost = purchase + operating + handling
total_purchase = machine_cost @ v + carrier_cost @ w # @ is dot product
total_oper     = cp.sum(cp.multiply(x, C_op))
total_hand     = cp.sum(cp.multiply(y, C_hand))
objective = cp.Minimize(total_purchase + total_oper + total_hand)

prob = cp.Problem(objective, cons)

# Solve
prob.solve(solver=cp.MOSEK)

def iv(a):  # integer value
    return np.round(a).astype(int)

print("Status:", prob.status)
print(f"Optimal total cost: ${prob.value:.2f}")
print("Machines v (M1, M2, M3, M4):", iv(v.value))
print("Carriers w (MH1, MH2, MH3):", iv(w.value))


Status: optimal
Optimal total cost: $5460010.95
Machines v (M1..M4): [4 8 0 6]
Carriers w (MH1..MH3): [0 1 1]
